In [1]:
import os
from pyTigerGraph import TigerGraphConnection
from dotenv import load_dotenv

load_dotenv()
# We first create a connection to the database
host = os.environ["HOST"]
username = os.getenv("USERNAME", "tigergraph")
password = os.getenv("PASS", "tigergraph")
conn = TigerGraphConnection(
    host=host,
    username=username,
    password=password,
)

In [2]:
conn.graphname = "GraphRAG_pytgdocs"
conn.gsql("""CREATE GRAPH GraphRAG_pytgdocs()""")

'The graph GraphRAG_pytgdocs is created.'

In [4]:
_ = conn.getToken()

In [5]:
# And then add CoPilot's address to the connection. This address
# is the host's address where the CoPilot container is running.
conn.ai.configureCoPilotHost("http://localhost:8000")
conn.ai.initializeSupportAI()

{'host_name': 'https://algotesting.i.tgcloud.io',
 'schema_creation_status': '"Using graph \'GraphRAG_pytgdocs\'\\nSuccessfully created schema change jobs: [add_supportai_schema].\\nWARNING: When modifying the graph schema, reinstalling all affected queries is required, and the duration of this process may vary based on the number and complexity of the queries. To skip query reinstallation, you can run with the \'-N\' option, but manual reinstallation of queries will be necessary afterwards.\\nKick off schema change job add_supportai_schema\\nDoing schema change on graph \'GraphRAG_pytgdocs\' (current version: 0)\\nTrying to add local vertex \'DocumentChunk\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Document\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Concept\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Entity\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Relationship\' to the graph \'Grap

In [6]:
access = os.environ["AWS_ACCESS_KEY_ID"]
sec = os.environ["AWS_SECRET_ACCESS_KEY"]
res = conn.ai.createDocumentIngest(
    data_source="s3",
    data_source_config={"aws_access_key": access, "aws_secret_key": sec},
    loader_config={"doc_id_field": "url", "content_field": "content"},
    file_format="json",
)

In [15]:
conn.ai.runDocumentIngest(
    res["load_job_id"],
    res["data_source_id"],
    "s3://tg-documentation/pytg_current/pytg_current.jsonl",
)

{'job_name': 'load_documents_content_json_203b064024e3499ea41b876cc67a85cf',
 'job_id': 'GraphRAG_pytgdocs.load_documents_content_json_203b064024e3499ea41b876cc67a85cf.stream.SupportAI_GraphRAG_pytgdocs_5b098715edbd4c878f7425918eb553c0.1721853566538',
 'log_location': '/home/tigergraph/tigergraph/log/kafkaLoader/GraphRAG_pytgdocs.load_documents_content_json_203b064024e3499ea41b876cc67a85cf.stream.SupportAI_GraphRAG_pytgdocs_5b098715edbd4c878f7425918eb553c0.1721853566538'}

In [ ]:
asdf

In [16]:
for v in ["Document", "Content", "DocumentChunk"]:
    try:
        conn.delVertices(v)
    except:
        pass

import time
time.sleep(3)
conn.ai.runDocumentIngest(
    res["load_job_id"],
    res["data_source_id"],
    "s3://tg-documentation/pytg_current/pytg_current.jsonl",
)

{'job_name': 'load_documents_content_json_203b064024e3499ea41b876cc67a85cf',
 'job_id': 'GraphRAG_pytgdocs.load_documents_content_json_203b064024e3499ea41b876cc67a85cf.stream.SupportAI_GraphRAG_pytgdocs_5b098715edbd4c878f7425918eb553c0.1721853623658',
 'log_location': '/home/tigergraph/tigergraph/log/kafkaLoader/GraphRAG_pytgdocs.load_documents_content_json_203b064024e3499ea41b876cc67a85cf.stream.SupportAI_GraphRAG_pytgdocs_5b098715edbd4c878f7425918eb553c0.1721853623658'}

In [ ]:
import httpx
import base64

# conn.ai.forceConsistencyUpdate()
# url = self.nlqs_host+"/"+self.conn.graphname+"/supportai/forceupdate"
# return self.conn._req("GET", url, authMode="pwd", resKey=None)
httpx.get(f"http://localhost:8000/{conn.graphname}/supportai/forceupdate")

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
print(model.invoke("hi"))
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

In [ ]:
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
import os

# from langchain_core.pydantic_v1 import BaseModel
from pydantic import BaseModel


class AnswerWithJustification(BaseModel):
    """An answer to the user question along with justification for the answer."""

    answer: str
    justification: str


os.environ["OPENAI_API_KEY"] = ""
model_name = "gpt-4o-mini"
llm = ChatOpenAI(model=model_name, temperature=0)
# sllm = llm.with_structured_output(AnswerWithJustification)
# print(sllm.invoke("What weighs more a pound of bricks or a pound of feathers"))


class GraphExtractor:
    def __init__(self):
        self.transformer = LLMGraphTransformer(
            llm=llm,
            node_properties=["description"],
            relationship_properties=["description"],
        )

    def extract(self, text):
        doc = Document(page_content=text)
        graph_docs = self.transformer.convert_to_graph_documents([doc])
        return graph_docs

In [ ]:
text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
ge = GraphExtractor()

docs = ge.extract(text)
for d in docs:
    for n in d.nodes:
        print(n)
    for r in d.relationships:
        print(r)
# print(f"Nodes:{docs[0].nodes}")
# print(f"Relationships:{docs[0].relationships}")
# docs